In [1]:
import grpc
import tensorflow as tf
import numpy as np

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

from keras_image_helper import create_preprocessor

Мы импортировали:
- `grpc` для связи с TF Serving
- `tensorflow` для определений protobuf
- пару функций из TF Serving

In [2]:
# Определяем подключение к сервису:

host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

Предварительная обработка изображений:

In [3]:
from keras_image_helper import create_preprocessor

preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [4]:
# Преобразовываем в массив Numpy
url = "http://bit.ly/mlbookcamp-pants"
X = preprocessor.from_url(url)

print("Loaded data:", X)

Loaded data: [[[[-0.11372548 -0.15294117 -0.19999999]
   [-0.11372548 -0.15294117 -0.19999999]
   [-0.10588235 -0.14509803 -0.19215685]
   ...
   [-0.01960784 -0.01960784 -0.08235294]
   [-0.04313725 -0.04313725 -0.10588235]
   [-0.11372548 -0.11372548 -0.17647058]]

  [[-0.09019607 -0.12941176 -0.17647058]
   [-0.09019607 -0.12941176 -0.17647058]
   [-0.08235294 -0.12156862 -0.16862744]
   ...
   [-0.01960784 -0.01960784 -0.08235294]
   [-0.04313725 -0.04313725 -0.10588235]
   [-0.10588235 -0.10588235 -0.16862744]]

  [[-0.09803921 -0.1372549  -0.18431371]
   [-0.09803921 -0.1372549  -0.18431371]
   [-0.09019607 -0.12941176 -0.17647058]
   ...
   [-0.01960784 -0.01960784 -0.08235294]
   [-0.03529412 -0.03529412 -0.09803921]
   [-0.09019607 -0.09019607 -0.15294117]]

  ...

  [[-0.67058825 -0.7019608  -0.7254902 ]
   [-0.6862745  -0.7176471  -0.7411765 ]
   [-0.70980394 -0.7411765  -0.7647059 ]
   ...
   [-0.62352943 -0.84313726 -0.9529412 ]
   [-0.6313726  -0.8509804  -0.9607843 ]
   

In [5]:
print("Type of X:", type(X))
print("Shape of X:", X.shape)
print("Dtype of X:", X.dtype)

Type of X: <class 'numpy.ndarray'>
Shape of X: (1, 299, 299, 3)
Dtype of X: float32


In [6]:
# Для gRPC нам необходимо преобразовать массив Numpy в protobuf c помощью специальной функции:

def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [7]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_layer_34'].CopyFrom(np_to_protobuf(X))

In [8]:
# Отправка запроса и получение результата
try:
    pb_result = stub.Predict(pb_request, timeout=20.0)
    print("Prediction result:", pb_result)
except grpc.RpcError as e:
    print("gRPC error:", e.code())
    print("Details:", e.details())

Prediction result: model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}
outputs {
  key: "output_0"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -0.19749479
    float_val: -3.24904633
    float_val: -0.616598964
    float_val: 0.224490911
    float_val: 9.35398865
    float_val: -0.603306174
    float_val: -3.84356332
    float_val: 4.89045572
    float_val: -0.379429489
    float_val: -3.13481736
  }
}



In [10]:
# Делаем прогноз:

pred = pb_result.outputs['output_0'].float_val

In [11]:
# Переводим в более читабельный формат:

labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
] 

result = {c: p for c, p in zip(labels, pred)}

In [12]:
result

{'dress': -0.19749478995800018,
 'hat': -3.2490463256835938,
 'longsleeve': -0.6165989637374878,
 'outwear': 0.2244909107685089,
 'pants': 9.353988647460938,
 'shirt': -0.6033061742782593,
 'shoes': -3.8435633182525635,
 'shorts': 4.890455722808838,
 'skirt': -0.37942948937416077,
 't-shirt': -3.134817361831665}